# Проект "Выгрузка данных для анализа из базы данных сайта StackOverFlow"


**Описание проекта**

Вы будете работать с базой данных **StackOverflow** — сервиса вопросов и ответов о программировании. 

**Цель проекта**: закрепить и продемонстрировать навыки владения языком SQL.

**Задача проекта**: выгрузить из базы необходимые наборы данных.

**План работы**

1. Решить задания первой части.
2. Решить задания второй части
3. Сделать вывод.  

**Используемые инструменты**

- PostgreSQL

**Описание данных**

[ER-диаграмма и описание базы данных](https://github.com/AVRotaev/Portfolio/blob/main/Stackoverflow_service_analysis_Data_export_PostgreSQL/ER-diagram_stackoverflow.pdf)


## Часть 1

#### Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
SELECT COUNT(p.id) AS top_questions_amt
FROM stackoverflow.posts p 
    JOIN stackoverflow.post_types t ON p.post_type_id = t.id
WHERE t.type = 'Question' 
    AND (p.score > 300 OR p.favorites_count >=100);

#### Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
SELECT ROUND(AVG(daily_posts_num))
FROM (SELECT DISTINCT DATE_TRUNC('day',p.creation_date)::date AS day,
        COUNT(*) OVER(PARTITION BY DATE_TRUNC('day',p.creation_date)::date) AS daily_posts_num
      FROM stackoverflow.posts p 
      JOIN stackoverflow.post_types pt ON p.post_type_id=pt.id
      WHERE pt.type='Question' 
          AND DATE_TRUNC('day',p.creation_date)::date BETWEEN '2008-11-01' AND '2008-11-18') AS tab
;

#### Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
SELECT COUNT(*)
FROM (SELECT DISTINCT user_id, 
            u.creation_date::date AS u_date, 
            b.creation_date::date AS b_date
    FROM stackoverflow.users u 
    JOIN stackoverflow.badges b ON u.id=b.user_id
    WHERE u.creation_date::date = b.creation_date::date
    ORDER BY user_id) AS tab
;

#### Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.users u 
    JOIN stackoverflow.posts p ON u.id = p.user_id
    JOIN stackoverflow.votes v ON p.id = v.post_id
WHERE u.display_name = 'Joel Coehoorn'  
;

#### Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
SELECT *, 
    RANK() OVER(ORDER BY id DESC) AS rank
FROM stackoverflow.vote_types
ORDER BY id

#### Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
SELECT DISTINCT u.id, 
       COUNT(v.id) as votes
FROM stackoverflow.users u 
    JOIN stackoverflow.votes v ON u.id = v.user_id
WHERE v.vote_type_id IN (SELECT id
                         FROM stackoverflow.vote_types
                          WHERE name = 'Close') 
GROUP BY u.id
ORDER BY votes desc, 
         u.id desc
LIMIT 10
;

#### Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.

Отобразите несколько полей:

- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.

Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
SELECT u.id, COUNT(b.creation_date) AS badges_count, 
        DENSE_RANK() OVER(ORDER BY COUNT(b.creation_date) DESC)
FROM stackoverflow.users u JOIN stackoverflow.badges b ON u.id = b.user_id        
WHERE b.creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY u.id
ORDER BY badges_count DESC, u.id
LIMIT 10
;

#### Сколько в среднем очков получает пост каждого пользователя?

In [ ]:
SELECT p.title AS post, u.id AS users, 
       p.score,
       ROUND(AVG(p.score) OVER(PARTITION BY u.id)) AS avg_votes
FROM stackoverflow.posts p JOIN stackoverflow.users u ON p.user_id = u.id                
WHERE NOT p.score = 0
      AND NOT p.title IS NULL

;

#### Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
WITH usr AS (SELECT u.id, COUNT(b.id) AS badges
            FROM stackoverflow.users u 
            JOIN stackoverflow.badges b ON u.id = b.user_id
            GROUP BY u.id 
            HAVING COUNT(b.id) > 1000)

SELECT p.title
FROM stackoverflow.posts p 
JOIN usr u ON p.user_id = u.id
WHERE p.title IS NOT NULL
;

#### Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:

- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

In [ ]:
SELECT id, views,
        CASE 
            WHEN views >= 350 THEN 1
            WHEN views >= 100 THEN 2
            ELSE 3
        END
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views != 0
;

#### Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. 

Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
WITH tab AS (SELECT id, views AS u_views,
                    CASE 
                        WHEN views >= 350 THEN 1
                        WHEN views >= 100 THEN 2
                        ELSE 3
                    END AS usr_group
            FROM stackoverflow.users
            WHERE location LIKE '%Canada%' AND views != 0),
            
tab1 AS (SELECT *,
                MAX(u_views) OVER(PARTITION BY usr_group) AS max_views
        FROM tab)
        
SELECT id, 
        usr_group, 
        u_views   
FROM tab1
WHERE u_views = max_views
ORDER BY u_views DESC, id
;

#### Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. 

Сформируйте таблицу с полями:

- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [ ]:
WITH usr AS (SELECT EXTRACT(DAY FROM creation_date) day_num, 
                     COUNT(id) AS users
             FROM stackoverflow.users
             WHERE creation_date::date BETWEEN '2008-11-01' AND '2008-11-30'
             GROUP BY day_num
            )

SELECT day_num, 
        users, 
        SUM(users) OVER(ORDER BY day_num)
FROM usr        
;

#### Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. 

Отобразите:

- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [ ]:
WITH tab AS (SELECT u.id AS user_id, 
                     u.creation_date AS reg_date,  
                     p.id AS post_id, 
                     p.creation_date AS post_date,
                     RANK() OVER(PARTITION BY u.id ORDER BY p.creation_date)
             FROM stackoverflow.users u 
             JOIN stackoverflow.posts p ON u.id = p.user_id
            )
            
SELECT user_id,
       post_date - reg_date
FROM tab    
WHERE rank = 1
;

## Часть 2

#### Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
SELECT DATE_TRUNC('month', creation_date)::date AS month, 
        SUM(views_count)  AS views_monthly
FROM stackoverflow.posts
GROUP BY month
ORDER BY views_monthly DESC

#### Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. 

Вопросы, которые задавали пользователи, не учитывайте. 

Для каждого имени пользователя выведите количество уникальных значений user_id. 

Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
SELECT u.display_name,
       COUNT(DISTINCT user_id)
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
JOIN stackoverflow.users u ON u.id = p.user_id
WHERE DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date)
  AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'
  AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(*) > 100
ORDER BY display_name
;

#### Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
WITH usr AS (SELECT u.id, COUNT(*)
            FROM stackoverflow.users u JOIN stackoverflow.posts p ON u.id = p.user_id
            WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01' 
                 AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01'
            GROUP BY u.id
            HAVING COUNT(*) > 0)
            
SELECT DATE_TRUNC('month', p.creation_date)::date AS month,
        COUNT(p.id) AS posts_num
FROM stackoverflow.posts p 
JOIN usr u ON p.user_id = u.id 
GROUP BY month
ORDER BY month DESC
;

#### Используя данные о постах, выведите несколько полей:

- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
SELECT u.id, 
        p.creation_date, 
        p.views_count,
        SUM(p.views_count) OVER(PARTITION BY u.id ORDER BY p.creation_date)
FROM stackoverflow.users u 
JOIN stackoverflow.posts p ON u.id = p.user_id
ORDER BY u.id
;

#### Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? 

Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. 

In [ ]:
WITH d AS (SELECT u.id, COUNT(DISTINCT p.creation_date::date) AS days
          FROM stackoverflow.users u 
          JOIN stackoverflow.posts p ON u.id = p.user_id
          WHERE p.creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'
          GROUP BY u.id)
          
SELECT ROUND(AVG(days))
FROM d
;

#### На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:

- Номер месяца.
- Количество постов за месяц.
- Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.



In [ ]:
SELECT *,
        ROUND((monthly_posts_amt * 100 / (LAG(monthly_posts_amt) OVER(ORDER BY month)::numeric)) - 100, 2) AS posts_delta
FROM (SELECT EXTRACT(MONTH FROM creation_date) AS month, 
            COUNT(*) AS monthly_posts_amt
    FROM stackoverflow.posts
    WHERE DATE_TRUNC('day', creation_date)::date BETWEEN '2008-09-01' AND '2008-12-31'
    GROUP BY month) tab

;    

#### Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:

- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
WITH posts AS (SELECT  EXTRACT(WEEK FROM p.creation_date) AS week_num,
                    p.creation_date,
                    ROW_NUMBER() OVER(PARTITION BY DATE_TRUNC('week', p.creation_date)::date ORDER BY p.creation_date) AS last_post_num
                   
                FROM (SELECT u.id, 
                          COUNt(p.id) AS posts_num
                      FROM stackoverflow.users u JOIN stackoverflow.posts p ON u.id = p.user_id
                      GROUP BY u.id
                      ORDER BY posts_num DESC
                      LIMIT 1) usr 
                JOIN stackoverflow.posts p ON usr.id = p.user_id
                WHERE DATE_TRUNC('day', p.creation_date) BETWEEN '2008-10-01' AND '2008-10-31')
                
SELECT week_num,
        creation_date
FROM (SELECT *,
        MAX(last_post_num) OVER(PARTITION BY week_num)
        FROM posts) tab
WHERE  last_post_num = max
;

## Общий вывод 

**Цель проекта**: закрепить и продемонстрировать навыки владения языком SQL.

**Задача проекта**: выгрузить из базы необходимые наборы данных.

**Вывод**: данные успешно выгружены и готовы к дальнейшей обработке и анализу.